In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet
from tensorflow.keras.applications.resnet50 import preprocess_input

import glob

#need to get 
import os
import shutil

from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications.resnet import ResNet152

from keras.applications.resnet import ResNet50
from keras.models import Model
import keras

import tensorflow.keras.applications.densenet as Densenet
from tensorflow.keras.applications.densenet import preprocess_input

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 22:47:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Trying random undersampling

### Make new directory

In [54]:
!mkdir ./OGNetDevelopmentData-2/train_RUS

In [55]:
!mkdir ./OGNetDevelopmentData-2/train_RUS/class_0
!mkdir ./OGNetDevelopmentData-2/train_RUS/class_1

In [56]:
#https://stackoverflow.com/questions/11372054/randomly-choose-100-documents-under-a-directory
import shutil, random, os
dirpath = '/content/OGNetDevelopmentData-2/train_images/class_0'
destDirectory = '/content/OGNetDevelopmentData-2/train_RUS'

#randomly select subset from negative group
filenames = random.sample(os.listdir(dirpath), 986)
for fname in filenames:
    srcpath = os.path.join(dirpath, fname)
    shutil.copy(srcpath, destDirectory)

In [57]:
!ls /content/OGNetDevelopmentData-2/train_RUS | wc -l

988


In [58]:
path = "/content/OGNetDevelopmentData-2/train_RUS/*_0.png"
destination = "/content/OGNetDevelopmentData-2/train_RUS/class_0"
files = glob.glob(path)

for f in files:
    shutil.move(f, destination)

In [59]:
 # Source path with original set of images
source = "/content/OGNetDevelopmentData-2/train_images/class_1/*_1.png"
  
# Destination path for 
destination = "/content/OGNetDevelopmentData-2/train_RUS/class_1"
files = glob.glob(source)

# Copy normal images to destination folder

for f in files:
  shutil.copy(f, destination)

In [60]:
!ls /content/OGNetDevelopmentData-2/train_RUS/class_0 | wc -l

986


In [61]:
!ls /content/OGNetDevelopmentData-2/train_RUS/class_1 | wc -l

116


### Loading in images

In [62]:
batch_size = 32
img_height = 500
img_width = 500

In [63]:
#training
#this is not a dataframe, it's some sort of an object that points to where the data lives (behaving more like generator)
#https://www.tensorflow.org/tutorials/images/classification
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/OGNetDevelopmentData-2/train_RUS',
  #validation_split=0.2,
  #subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1102 files belonging to 2 classes.


In [64]:
#validation
#https://www.tensorflow.org/tutorials/images/classification
val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/OGNetDevelopmentData-2/val_images',
  #validation_split=0.2,
  #subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 708 files belonging to 2 classes.


In [65]:
#testing
#https://www.tensorflow.org/tutorials/images/classification
test_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/OGNetDevelopmentData-2/test_images',
  #validation_split=0.2,
  #subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 704 files belonging to 2 classes.


In [66]:
size=(224,224)

In [67]:
#from tensorflow
from tensorflow.image import resize
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))


### ResNet50 (Random undersampling)

In [68]:
#https://www.tensorflow.org/tutorials/images/classification
AUTOTUNE = tf.data.AUTOTUNE

train_ds_prefetch = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_prefetch = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds_prefetch = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [69]:
#Resnet50 preprocessing (The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.)
resnet_preproc_layer = tf.keras.applications.resnet50.preprocess_input
resnet_train_ds = train_ds_prefetch.map(lambda x, y: (resnet_preproc_layer(x), y))
resnet_val_ds =val_ds_prefetch.map(lambda x, y: (resnet_preproc_layer(x), y))
resnet_test_ds = test_ds_prefetch.map(lambda x, y: (resnet_preproc_layer(x), y))

In [70]:
normalization_layer = layers.Rescaling(1./224)
normalized_train_ds = resnet_train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds =resnet_val_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds = resnet_test_ds.map(lambda x, y: (normalization_layer(x), y))

In [71]:
#setup ResNet50
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

output = restnet.layers[-1].output #how does layer work if output isn't defined yet?
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output)

for layer in restnet.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [72]:
model = Sequential()
model.add(restnet)
model.add(Dense(1, activation='sigmoid'))

In [73]:
#minimal data augmentation
data_augmentation_adam = keras.Sequential(
    [keras.layers.RandomFlip("horizontal"), keras.layers.RandomRotation(0.1),]
)

In [74]:
resnet_train_ds_aug_adam = train_ds_prefetch.map(lambda x, y: (data_augmentation_adam(x), y))


In [75]:
resnet_train_ds_aug_adam = resnet_train_ds_aug_adam.map(lambda x, y: (resnet_preproc_layer(x), y))
normalized_train_ds_aug_adam = resnet_train_ds_aug_adam.map(lambda x, y: (normalization_layer(x), y))

In [76]:
#method of calculating loss, optimization (optimization algorithm that provides more efficient neural network weights by running repeated cycles of ADAM-  “adaptive moment estimation.”), determining metrics
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall(), keras.metrics.Precision()])

In [77]:
#subpar results here
repeat_history_plain = model.fit(normalized_train_ds,
                    epochs=5, 
                    validation_data=normalized_val_ds) 
                    

Epoch 1/5
35/35 [==============================] - 22s 291ms/step - loss: 0.9852 - binary_accuracy: 0.8403 - recall: 0.1121 - precision: 0.1512 - val_loss: 0.1777 - val_binary_accuracy: 0.9718 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/5
35/35 [==============================] - 5s 152ms/step - loss: 0.3214 - binary_accuracy: 0.8993 - recall: 0.0776 - precision: 0.6923 - val_loss: 0.1653 - val_binary_accuracy: 0.9732 - val_recall: 0.0667 - val_precision: 0.1667
Epoch 3/5
35/35 [==============================] - 5s 152ms/step - loss: 0.2932 - binary_accuracy: 0.9056 - recall: 0.1466 - precision: 0.7727 - val_loss: 0.2245 - val_binary_accuracy: 0.9689 - val_recall: 0.0667 - val_precision: 0.1111
Epoch 4/5
35/35 [==============================] - 5s 151ms/step - loss: 0.2957 - binary_accuracy: 0.9020 - recall: 0.1552 - precision: 0.6429 - val_loss: 0.2497 - val_binary_accuracy: 0.9534 - val_recall: 0.3333 - val_precision: 0.1786
Epoch 5/5
35/35 [==========================

In [ ]:
model.save('/content/gdrive/MyDrive/RUS_resnet_correct')
model.save('/content/gdrive/MyDrive/RUS_resnet_correct.h5')

In [ ]:
model_RUS=load_model('/content/gdrive/MyDrive/RUS_resnet_correct')

In [ ]:
#trying early stopping with patience of 5 cycles
es2 = EarlyStopping(monitor='val_recall', mode='min', verbose=1, patience=5)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall(), keras.metrics.Precision()])

In [ ]:
more_epochs=model_RUS.fit(normalized_train_ds,
                    epochs=30, 
                    validation_data=normalized_val_ds) #why does the training drop after 11 epochs? 

In [78]:
data_augmentation_2 = keras.Sequential(
    [keras.layers.RandomFlip("horizontal"), keras.layers.RandomRotation(0.2),
     keras.layers.RandomContrast((0.1,0.1))]
)

In [79]:
#using new data augmentation from above 
resnet_train_ds_aug2 = train_ds_prefetch.map(lambda x, y: (data_augmentation_2(x), y))


In [80]:
resnet_train_ds_aug2 = resnet_train_ds_aug2.map(lambda x, y: (resnet_preproc_layer(x), y))
normalized_train_ds_aug2 = resnet_train_ds_aug2.map(lambda x, y: (normalization_layer(x), y))

In [81]:
repeat_history2 = model.fit(normalized_train_ds_aug2, #it's getting stuck now at recall=0.2 or 0.13 with greater (0.5, 0.3) vs. less RandomRotation and contrast (0.2, 0.1)
                    epochs=5, 
                    validation_data=normalized_val_ds) 
#I guess that means it's overfitting?

Epoch 1/5
35/35 [==============================] - 6s 159ms/step - loss: 0.2736 - binary_accuracy: 0.9065 - recall: 0.2155 - precision: 0.6757 - val_loss: 0.2330 - val_binary_accuracy: 0.9449 - val_recall: 0.3333 - val_precision: 0.1471
Epoch 2/5
35/35 [==============================] - 5s 152ms/step - loss: 0.2644 - binary_accuracy: 0.9083 - recall: 0.2328 - precision: 0.6923 - val_loss: 0.2268 - val_binary_accuracy: 0.9449 - val_recall: 0.3333 - val_precision: 0.1471
Epoch 3/5
35/35 [==============================] - 5s 152ms/step - loss: 0.2559 - binary_accuracy: 0.9138 - recall: 0.2672 - precision: 0.7561 - val_loss: 0.2188 - val_binary_accuracy: 0.9492 - val_recall: 0.3333 - val_precision: 0.1613
Epoch 4/5
35/35 [==============================] - 6s 158ms/step - loss: 0.2472 - binary_accuracy: 0.9183 - recall: 0.3017 - precision: 0.7955 - val_loss: 0.2081 - val_binary_accuracy: 0.9492 - val_recall: 0.2667 - val_precision: 0.1379
Epoch 5/5
35/35 [==============================] - 5

In [ ]:

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

#output = restnet.layers[-1].output #how does layer work if output isn't defined yet?
#output = keras.layers.Flatten()(output)
#restnet = Model(restnet.input, output)

for layer in restnet.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(restnet)
model.add(GlobalAveragePooling2D())
#model.add(Dense(512, activation='relu', input_dim=(None, 100352)))
#model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall(), keras.metrics.Precision()])

In [ ]:
repeat_history3 = model.fit(normalized_train_ds_aug_adam, #global average pooling cause total miss of recall/precision
                    epochs=10, 
                    validation_data=normalized_val_ds) 

### ResNet152 (w/Random undersampling)

In [82]:
restnet = ResNet152(include_top=False, weights='imagenet', input_shape=(224,224,3))

output = restnet.layers[-1].output #the output is a method of resnet.layers (also called output which is confusing)
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output)

for layer in restnet.layers:
    layer.trainable = False

234698864/234698864 [==============================] - 2s 0us/step


In [83]:
model = Sequential()
model.add(restnet)
model.add(Dense(1, activation='sigmoid'))

In [84]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall(), keras.metrics.Precision()])

In [85]:
repeat_history3 = model.fit(normalized_train_ds_aug2, #it's getting stuck now at recall=0.2 or 0.13 with greater (0.5, 0.3) vs. less RandomRotation and brightness (0.2, 0.1)
                    epochs=5, 
                    validation_data=normalized_val_ds) 

Epoch 1/5
35/35 [==============================] - 22s 421ms/step - loss: 0.8197 - binary_accuracy: 0.8376 - recall_1: 0.0603 - precision_1: 0.0909 - val_loss: 0.2932 - val_binary_accuracy: 0.9350 - val_recall_1: 0.1333 - val_precision_1: 0.0571
Epoch 2/5
35/35 [==============================] - 12s 350ms/step - loss: 0.3144 - binary_accuracy: 0.8920 - recall_1: 0.0431 - precision_1: 0.3846 - val_loss: 0.1548 - val_binary_accuracy: 0.9760 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/5
35/35 [==============================] - 12s 353ms/step - loss: 0.2905 - binary_accuracy: 0.8929 - recall_1: 0.0345 - precision_1: 0.4000 - val_loss: 0.1937 - val_binary_accuracy: 0.9732 - val_recall_1: 0.1333 - val_precision_1: 0.2500
Epoch 4/5
35/35 [==============================] - 12s 353ms/step - loss: 0.2806 - binary_accuracy: 0.8966 - recall_1: 0.0690 - precision_1: 0.5714 - val_loss: 0.2337 - val_binary_accuracy: 0.9605 - val_recall_1: 0.1333 - val_precision_1: 0.1176
Epoch 5/

### Xception (w/Random undersampling)

In [86]:
base_model = keras.applications.Xception(include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

for layer in base_model.layers: 
    layer.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [87]:
full_model = Sequential()
full_model.add(base_model)
full_model.add(Flatten())
#full_model.add(Dense(512, activation='relu'))
#full_model.add(Dropout(0.3))
#full_model.add(Dense(512, activation='relu'))
#full_model.add(Dropout(0.3))
full_model.add(Dense(1, activation='sigmoid'))

In [88]:
full_model.compile(optimizer=keras.optimizers.Adam(), loss='binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy(),keras.metrics.Recall(), keras.metrics.Precision()])

In [89]:
xcept_preproc_layer = tf.keras.applications.xception.preprocess_input
xcept_train_ds = train_ds_prefetch.map(lambda x, y: (xcept_preproc_layer(x), y))
xcept_val_ds =val_ds_prefetch.map(lambda x, y: (xcept_preproc_layer(x), y))
xcept_test_ds = test_ds_prefetch.map(lambda x, y: (xcept_preproc_layer(x), y))

In [90]:
normalization_layer = layers.Rescaling(1./224, offset=-1)
normalized_train_ds_xc = xcept_train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds_xc =xcept_val_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds_xc = xcept_test_ds.map(lambda x, y: (normalization_layer(x), y))

In [91]:
history_xcept_1 = full_model.fit(normalized_train_ds_xc, 
                    epochs=10, 
                    validation_data=normalized_val_ds_xc) 

Epoch 1/10
35/35 [==============================] - 12s 255ms/step - loss: 0.3776 - binary_accuracy: 0.8711 - recall_2: 0.0259 - precision_2: 0.0938 - val_loss: 0.1716 - val_binary_accuracy: 0.9788 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 2/10
35/35 [==============================] - 7s 196ms/step - loss: 0.3293 - binary_accuracy: 0.8947 - recall_2: 0.0000e+00 - precision_2: 0.0000e+00 - val_loss: 0.1682 - val_binary_accuracy: 0.9788 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 3/10
35/35 [==============================] - 7s 197ms/step - loss: 0.3194 - binary_accuracy: 0.8947 - recall_2: 0.0000e+00 - precision_2: 0.0000e+00 - val_loss: 0.1858 - val_binary_accuracy: 0.9788 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 4/10
35/35 [==============================] - 7s 198ms/step - loss: 0.3126 - binary_accuracy: 0.8956 - recall_2: 0.0086 - precision_2: 1.0000 - val_loss: 0.1868 - val_binary_accuracy: 0.9788 - val_recall_2: 0.0000e+

KeyboardInterrupt: ignored

In [108]:
data_augmentation_adam = keras.Sequential(
    [keras.layers.RandomFlip("horizontal"), keras.layers.RandomRotation(0.1),]
)

In [109]:
xcept_train_ds_aug_adam = train_ds_prefetch.map(lambda x, y: (data_augmentation_adam(x), y))


In [110]:
xcept_train_ds_aug_adam = resnet_train_ds_aug_adam.map(lambda x, y: (xcept_preproc_layer(x), y))
normalized_train_ds_xc_aug_adam = xcept_train_ds_aug_adam.map(lambda x, y: (normalization_layer(x), y))

In [111]:
history_xcept_1_aug = full_model.fit(normalized_train_ds_xc_aug_adam, 
                    epochs=10, 
                    validation_data=normalized_val_ds_xc) 

Epoch 1/10
35/35 [==============================] - 5s 151ms/step - loss: 0.3378 - binary_accuracy: 0.8875 - recall_3: 0.0000e+00 - precision_3: 0.0000e+00 - val_loss: 0.1751 - val_binary_accuracy: 0.9774 - val_recall_3: 0.0667 - val_precision_3: 0.3333
Epoch 2/10
35/35 [==============================] - 5s 149ms/step - loss: 0.2956 - binary_accuracy: 0.8966 - recall_3: 0.1034 - precision_3: 0.5455 - val_loss: 0.1279 - val_binary_accuracy: 0.9774 - val_recall_3: 0.0667 - val_precision_3: 0.3333
Epoch 3/10
35/35 [==============================] - 5s 149ms/step - loss: 0.2933 - binary_accuracy: 0.8966 - recall_3: 0.0862 - precision_3: 0.5556 - val_loss: 0.1271 - val_binary_accuracy: 0.9774 - val_recall_3: 0.0667 - val_precision_3: 0.3333
Epoch 4/10
35/35 [==============================] - 5s 150ms/step - loss: 0.2902 - binary_accuracy: 0.8984 - recall_3: 0.1034 - precision_3: 0.6000 - val_loss: 0.1211 - val_binary_accuracy: 0.9760 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00


KeyboardInterrupt: ignored

### Densenets (w/Random undersampling)

In [93]:
desnet_preproc_layer = tf.keras.applications.densenet.preprocess_input
desnet_train_ds = train_ds_prefetch.map(lambda x, y: (desnet_preproc_layer(x), y))
desnet_val_ds =val_ds_prefetch.map(lambda x, y: (desnet_preproc_layer(x), y))
desnet_test_ds = test_ds_prefetch.map(lambda x, y: (desnet_preproc_layer(x), y))

In [94]:
normalization_layer = layers.Rescaling(1./224, offset=-1)
normalized_train_ds_des = desnet_train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds_des =desnet_val_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds_des = desnet_test_ds.map(lambda x, y: (normalization_layer(x), y))

In [95]:
densenet_model= tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

for layer in densenet_model.layers: 
    layer.trainable = False

29084464/29084464 [==============================] - 0s 0us/step


In [96]:
full_model = Sequential()
full_model.add(densenet_model)
full_model.add(Flatten())
full_model.add(Dense(1,activation='sigmoid'))

In [ ]:
full_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 1)                 50177     
                                                                 
Total params: 7,087,681
Trainable params: 50,177
Non-trainable params: 7,037,504
_________________________________________________________________


In [97]:
full_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = [keras.metrics.BinaryAccuracy(),keras.metrics.Recall(), keras.metrics.Precision()])

In [98]:
data_augmentation_v2 = keras.Sequential(
    [keras.layers.RandomFlip("horizontal_and_vertical"), keras.layers.RandomRotation(0.2), keras.layers.RandomContrast(0.2)]
)

In [101]:
desnet_train_ds_aug_v2 = train_ds_prefetch.map(lambda x, y: (data_augmentation_v2(x), y))


In [102]:
desnet_train_ds_aug_v2 = desnet_train_ds_aug_v2.map(lambda x, y: (desnet_preproc_layer(x), y))
normalized_train_ds_des_aug_v2 = desnet_train_ds_aug_v2.map(lambda x, y: (normalization_layer(x), y))

In [105]:
history_dense_1_augv1 = full_model.fit(normalized_train_ds_des_aug_v2,
                    epochs=5, 
                    validation_data=normalized_val_ds_des) #still not much improvement

Epoch 1/5
35/35 [==============================] - 5s 152ms/step - loss: 0.2309 - binary_accuracy: 0.9192 - recall_3: 0.2586 - precision_3: 0.9091 - val_loss: 0.0885 - val_binary_accuracy: 0.9760 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 2/5
35/35 [==============================] - 5s 151ms/step - loss: 0.2200 - binary_accuracy: 0.9265 - recall_3: 0.3276 - precision_3: 0.9268 - val_loss: 0.0884 - val_binary_accuracy: 0.9774 - val_recall_3: 0.0667 - val_precision_3: 0.3333
Epoch 3/5
35/35 [==============================] - 5s 150ms/step - loss: 0.2105 - binary_accuracy: 0.9292 - recall_3: 0.3534 - precision_3: 0.9318 - val_loss: 0.0888 - val_binary_accuracy: 0.9774 - val_recall_3: 0.0667 - val_precision_3: 0.3333
Epoch 4/5
35/35 [==============================] - 5s 151ms/step - loss: 0.2018 - binary_accuracy: 0.9328 - recall_3: 0.3879 - precision_3: 0.9375 - val_loss: 0.0894 - val_binary_accuracy: 0.9788 - val_recall_3: 0.1333 - val_precision_3: 0.5000
Epoch 5/5
35